In [ ]:
! pip install bcc==0.1.10


In [1]:
%pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement bcc==0.29.1 (from versions: 0.1.7, 0.1.8, 0.1.10)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for bcc==0.29.1



  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached Babel-2.10.3-py3-none-any.whl.metadata (1.3 kB)


In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from dotenv import load_dotenv


In [2]:
from pdfminer.high_level import extract_text
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from transformers import pipeline
import wave
import numpy as np
from IPython.display import Audio
import sounddevice as sd
import os
from scipy.io.wavfile import write
import openai
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage

d:\D-Program Files\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
PDF_FILE = "data\hashir-ayaz-resume.pdf"

# ! Prompt to question till llm has asked 5 questions
PROMPT_TEMPLATE = """
You are an expert AI interviewer. You have technical knowledge of whatever domain the user is interviewing for.
Your task is to carefully review the experiences and projects provided in the context below, focusing on any technical details, challenges, or skills used.
Based on this review, generate insightful, technically focused questions to ask the individual about their experience.
You start by introducing the candidate to themselves and then ask a question.
Maintaining a conversational tone is important.
IMPORTANT POINTS:

Make sure the question is related to technical skills required in the Job Position provided below.
Focus on the Experiences and Projects section in the Context provided below.
Frame the question in a conversational manner that is easy to understand and answer.
After a question is answered, you should ask a follow up question to the same question if it is not answered properly or if the answer is not detailed.
If the answer is closely related to the job position, you should probe deeper into the answer.

IMPORTANT: Only follow up a question, atmost 2 times. If the user does not answer properly even after 2 times, proceed with the next question.
Ask a total of 5 question and then conclude the conversation by saying "Good Bye" inorder to clarify that the interview has been concluded.


Job Position: {field}
Context: {context}

Return the question as a single string, without introductory phrases like "Here's a question for you."

### Example question ### : "Imagine you're working on a web application with a large dataset of user interactions, and you need to display this data 
dynamically in the React frontend. Describe how you would design the backend API to support efficient data retrieval and provide a responsive 
user experience. Specifically, explain how you would manage pagination, filtering, and sorting on the backend using Node.js and MongoDB."

"""


# ! Prompt to question till llm feels it is questioned enough
PROMPT_TEMPLATE0 = """
You are an expert AI interviewer. You have technical knowledge of whatever domain the user is interviewing for.
Your task is to carefully review the experiences and projects provided in the context below, focusing on any technical details, challenges, or skills used.
Based on this review, generate insightful, technically focused questions to ask the individual about their experience.
You start by introducing the candidate to themselves and then ask a question.
Maintaining a conversational tone is important.
IMPORTANT POINTS:

Make sure the question is related to technical skills required in the Job Position provided below.
Focus on the Experiences and Projects section in the Context provided below.
Frame the question in a conversational manner that is easy to understand and answer.
After a question is answered, you should ask a follow up question to the same question if it is not answered properly or if the answer is not detailed.
If the answer is closely related to the job position, you should probe deeper into the answer.

IMPORTANT: Only follow up a question, atmost 2 times. If the user does not answer properly even after 2 times, proceed with the next question.
Ask questions until you feel you have questioned the candidate on a good amount of their data but make sure to not ask any more than 10 questions
and then conclude the conversation by saying "Good Bye" inorder to clarify that the interview has been concluded.


Job Position: {field}
Context: {context}

Return the question as a single string, without introductory phrases like "Here's a question for you."

### Example question ### : "Imagine you're working on a web application with a large dataset of user interactions, and you need to display this data 
dynamically in the React frontend. Describe how you would design the backend API to support efficient data retrieval and provide a responsive 
user experience. Specifically, explain how you would manage pagination, filtering, and sorting on the backend using Node.js and MongoDB."

"""


<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
C:\Users\HP\AppData\Local\Temp\ipykernel_3384\1383357857.py:2: SyntaxWarning: invalid escape sequence '\h'
  PDF_FILE = "data\hashir-ayaz-resume.pdf"


In [ ]:
class AI_Interviewer:
    def __init__(self):
        self.llm = None
        self.tts = None
        self.stt = None
        self.memory = None
  
    def create_llm(self, context, field):

        llm_used = ChatGroq(
            model="llama3-8b-8192",
            temperature=0.0,
            max_retries=2,
        )

        prompt = ChatPromptTemplate(
            messages=[
                # Type your Prompt
                SystemMessagePromptTemplate.from_template(
                    PROMPT_TEMPLATE.format(context=context, field=field)
                ),
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("""{text}""")
            ]
        )

        memory = ConversationSummaryBufferMemory( llm=llm_used, max_token_limit=3000, memory_key="chat_history", return_messages=True)
        conversation = LLMChain(llm=llm_used, prompt=prompt, verbose=False, memory=memory)

        return conversation

    def initialize_models(self, context, field):

        self.llm = self.create_llm(context, field)
        self.tts = pipeline("text-to-speech", model="facebook/mms-tts-eng")
        self.stt = pipeline(   "automatic-speech-recognition", model="openai/whisper-small")

    def read_csv(self, pdf_file):
        return extract_text(pdf_file)

    def invoke_llm(self, message):
        # Prepare the input for the conversation
    
        response = self.llm.invoke(message)

        question = response['text']
        print("Interviewer:")
        print(question)

        return question

    def invoke_tts(self, text, save_audio=False):
        audio = self.tts(text)

        if not save_audio:
            Audio(audio["audio"], rate=16000, autoplay=True)
        else:
            audio_data = audio["audio"]
            audio_data = (audio_data * 32767).astype(np.int16)
            sample_rate = 16000
            output_file = "audio.wav"

            with wave.open(output_file, "wb") as wf:
                wf.setnchannels(1)
                wf.setsampwidth(2)
                wf.setframerate(sample_rate)
                wf.writeframes(audio_data.tobytes())

            print(f"Audio saved to {output_file}")

    def invoke_stt(self, audio_file="audio.wav"):
        if not os.path.exists(audio_file) or os.path.getsize(audio_file) == 0:
            print("Error: audio file does not exist or is empty.")
            return None
        transcription = self.stt(audio_file)
        return transcription["text"]

    def hear_user(self, duration=5, sample_rate=16000):
        def record_audio(duration, sample_rate, device=None):
            try:
                print(f"Recording for {duration} seconds...")
                audio = sd.rec(
                    int(duration * sample_rate),
                    samplerate=sample_rate,
                    channels=1,
                    dtype=np.float32,
                    device=device,
                )
                sd.wait()
                print("Recording finished!")
                return audio
            except Exception as e:
                print("Error recording audio:", e)
                return None

        audio_data = record_audio(duration, sample_rate)
        write("audio.wav", sample_rate, audio_data)

    def generate_report(self, interview_responses, job):
        REPORT_TEMPLATE = """
        You are a recruiter and you want to hire the best candidate for the job position provided below.
        You are being provided with a list of questions and answers from an interview. Your task is to generate a report based on the interview responses. You have expertise in the field of the job position provided below. you will give the employee a rating out of 10 and a list of strengths and weaknesses.
        Job Position: {job}
        Interview Responses: {interview_responses}
        
        The report should be in the following format:
        {
            strengths: [list of strengths],
            weaknesses: [list of weaknesses],
            overall_rating: [rating out of 10],
            overall_feedback: [feedback],
        }
        """
        prompt_template = ChatPromptTemplate.from_template(REPORT_TEMPLATE)
        pass

    def start_interview(self, job):

        # Initialize interview variables and context
        interview_responses = []

        # Loop until the interview is complete
        while True:

            # Generate question using chat history
            current_question = self.invoke_llm("Hello")

            print(f"\nInterview Question: {current_question}")

            if "good bye" in current_question.lower():
                print("Ending interview based on user request.")
                break

            # Record and transcribe user's response
            print("\nRecording User's Answer:")
            self.hear_user(duration=20, sample_rate=16000)
            user_response = self.invoke_stt("audio.wav")
            print("\nUser Response:", user_response)

            # Append to interview responses
            interview_responses.append(
                {"question": current_question, "answer": user_response}
            )

            if "stop" in user_response.lower() :
                print("Ending interview based on user request.")
                break


        # Generate final report
        interview_summary = self.generate_report(interview_responses, job)

        print("\nInterview Summary:")
        print(interview_summary)

        return interview_responses, interview_summary
    
    

In [75]:

interviewer = AI_Interviewer()
cv_description = interviewer.read_csv(PDF_FILE)
interviewer.initialize_models(cv_description, "MERN developer")



Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [76]:
interviewer.start_interview(
    job="Mern stack developer",
)

Interviewer:
Hello Hashir! It's great to have you here today. I'm excited to learn more about your experience and skills as a MERN developer. Let's dive right in!

Can you tell me more about your experience with Docker and GitHub Actions in your current role as a Backend Engineering Intern at Cosmosys? How did you set up the continuous integration and deployment pipelines, and what kind of benefits did you see from implementing this process?

Interview Question: Hello Hashir! It's great to have you here today. I'm excited to learn more about your experience and skills as a MERN developer. Let's dive right in!

Can you tell me more about your experience with Docker and GitHub Actions in your current role as a Backend Engineering Intern at Cosmosys? How did you set up the continuous integration and deployment pipelines, and what kind of benefits did you see from implementing this process?

Recording User's Answer:
Recording for 20 seconds...
Recording finished!


d:\D-Program Files\Python\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input


User Response:  Yes, I did work on my project as a backend engineer. It was capable but I learned a lot of new things and a lot of new pipelines such as CIDI or CD pipelines and they helped me understand better
Interviewer:
Nice to meet you again, Hashir! So, you mentioned setting up a continuous integration and deployment pipeline using Docker and GitHub Actions, which reduced deployment time by 40%. That's impressive!

To drill down a bit further, can you walk me through the specific steps you took to set up the pipeline, and how exactly did you measure the 40% reduction in deployment time?

Interview Question: Nice to meet you again, Hashir! So, you mentioned setting up a continuous integration and deployment pipeline using Docker and GitHub Actions, which reduced deployment time by 40%. That's impressive!

To drill down a bit further, can you walk me through the specific steps you took to set up the pipeline, and how exactly did you measure the 40% reduction in deployment time?

R

d:\D-Program Files\Python\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



User Response:  I used were CI or CD pipeline and I also worked on MongoDB to integrate them and yes I did measure
Interviewer:
I think we might be getting a little stuck here, Hashir! It seems like we're not quite connecting on this question. To help move forward, can you try to provide a more detailed answer about the steps you took to set up the pipeline and how you measured the 40% reduction in deployment time?

Interview Question: I think we might be getting a little stuck here, Hashir! It seems like we're not quite connecting on this question. To help move forward, can you try to provide a more detailed answer about the steps you took to set up the pipeline and how you measured the 40% reduction in deployment time?

Recording User's Answer:
Recording for 20 seconds...
Recording finished!


...

d:\D-Program Files\Python\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



User Response:  With the Hodo Management app, I use MVCR architecture because it is very scalable and maintainable in the code base. Users can easily enter themselves and we can easily scale them to many more users and I use
Interviewer:
I think we've reached the end of our conversation, Hashir. It's been great speaking with you, but I'm not getting the level of detail I need to further explore your experience and skills.

It was a pleasure talking to you, and I hope our conversation has been helpful in some way. Good Bye!

Interview Question: I think we've reached the end of our conversation, Hashir. It's been great speaking with you, but I'm not getting the level of detail I need to further explore your experience and skills.

It was a pleasure talking to you, and I hope our conversation has been helpful in some way. Good Bye!

Recording User's Answer:
Recording for 20 seconds...
Recording finished!


d:\D-Program Files\Python\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



User Response:  Goodbye.
Ending interview based on user request.

Interview Summary:
None


([{'question': "Hello Hashir! It's great to have you here today. I'm excited to learn more about your experience and skills as a MERN developer. Let's dive right in!\n\nCan you tell me more about your experience with Docker and GitHub Actions in your current role as a Backend Engineering Intern at Cosmosys? How did you set up the continuous integration and deployment pipelines, and what kind of benefits did you see from implementing this process?",
   'answer': ' Yes, I did work on my project as a backend engineer. It was capable but I learned a lot of new things and a lot of new pipelines such as CIDI or CD pipelines and they helped me understand better'},
  {'question': "Nice to meet you again, Hashir! So, you mentioned setting up a continuous integration and deployment pipeline using Docker and GitHub Actions, which reduced deployment time by 40%. That's impressive!\n\nTo drill down a bit further, can you walk me through the specific steps you took to set up the pipeline, and how ex